In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from importlib import reload
import logging
reload(logging)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%H:%M:%S')

In [3]:
import sys

sys.path.insert(0, '../..')

## Load Dataset

In [4]:
from entity_embed.benchmarks import CompanyBenchmark

benchmark = CompanyBenchmark(data_dir_path="../data/")
benchmark

20:49:45 INFO:Extracting Company...
20:49:47 INFO:Reading Company row_dict...
20:49:49 INFO:Reading Company train.csv...
20:49:49 INFO:Reading Company valid.csv...
20:49:49 INFO:Reading Company test.csv...


<CompanyBenchmark> from http://pages.cs.wisc.edu/~anhai/data1/deepmatcher_data/Textual/Company/company_exp_data.zip

## Preprocess

In [5]:
attr_list = ['content']

In [6]:
from tqdm.auto import tqdm
import unidecode
import itertools
from entity_embed import default_tokenizer

def clean_str(s):
    s = unidecode.unidecode(s).lower().strip()
    s_tokens = itertools.islice((s_part[:30] for s_part in default_tokenizer(s)), 0, 1000)
    return ' '.join(s_tokens)

for row in tqdm(benchmark.row_dict.values()):
    for attr in attr_list:
        row[attr] = clean_str(row[attr])

## Init Data Module

In [7]:
import torch
import numpy as np

random_seed = 42
torch.manual_seed(random_seed)
np.random.seed(random_seed)

In [8]:
attr_info_dict = {
    'semantic_content': {
        'source_attr': 'content',
        'field_type': "SEMANTIC_MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'vocab': "fasttext.en.300d",
        'use_mask': True,
    },
}

In [9]:
from entity_embed import AttrInfoDictParser

row_numericalizer = AttrInfoDictParser.from_dict(attr_info_dict, row_dict=benchmark.row_dict)
row_numericalizer.attr_info_dict

20:50:29 INFO:Loading vectors from .vector_cache/wiki.en.vec.pt


{'semantic_content': NumericalizeInfo(source_attr='content', field_type=<FieldType.SEMANTIC_MULTITOKEN: 'semantic_multitoken'>, tokenizer='entity_embed.data_utils.numericalizer.default_tokenizer', alphabet=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', ' '], max_str_len=None, vocab=<torchtext.vocab.Vocab object at 0x7fdbc79f01f0>, n_channels=8, embed_dropout_p=0.2, use_attention=True, use_mask=True)}

In [10]:
datamodule = benchmark.build_datamodule(
    row_numericalizer=row_numericalizer,
    batch_size=20,
    row_batch_size=16,
    random_seed=random_seed
)

## Training

In [11]:
from entity_embed import LinkageEmbed
from pytorch_metric_learning import miners
from pytorch_metric_learning import losses

ann_k = 100
model = LinkageEmbed(
    datamodule,
    ann_k=ann_k,
    embedding_size=300,
    sim_threshold_list=[0.7, 0.8, 0.9],
)

In [12]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

max_epochs = 100
early_stop_callback = EarlyStopping(
   monitor='valid_f1_at_0.9',
   min_delta=0.00,
   patience=5,
   verbose=True,
   mode='max'
)
tb_log_dir = '../tb_logs'
tb_name = f'f1-{benchmark.dataset_name}'
trainer = pl.Trainer(
    gpus=1,
    max_epochs=max_epochs,
    check_val_every_n_epoch=1,
    callbacks=[early_stop_callback],
    logger=TensorBoardLogger(tb_log_dir, name=tb_name)
)

20:50:40 INFO:GPU available: True, used: True
20:50:40 INFO:TPU available: None, using: 0 TPU cores
20:50:40 INFO:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [13]:
trainer.fit(model, datamodule)

20:50:43 INFO:
  | Name        | Type       | Params
-------------------------------------------
0 | blocker_net | BlockerNet | 221 M 
1 | losser      | SupConLoss | 0     
-------------------------------------------
407 K     Trainable params
220 M     Non-trainable params
221 M     Total params


1

In [14]:
from entity_embed import validate_best

validate_best(trainer)

{'valid_f1_at_0.7': 0.24590435214297532,
 'valid_f1_at_0.8': 0.525905220048105,
 'valid_f1_at_0.9': 0.6042462467797814,
 'valid_pair_entity_ratio_at_0.7': 2.679968426591826,
 'valid_pair_entity_ratio_at_0.8': 0.8491492720575338,
 'valid_pair_entity_ratio_at_0.9': 0.4872829328188037,
 'valid_precision_at_0.7': 0.14589128513924796,
 'valid_precision_at_0.8': 0.4177855814914274,
 'valid_precision_at_0.9': 0.6121310295176385,
 'valid_recall_at_0.7': 0.7819680757761797,
 'valid_recall_at_0.8': 0.709524644799158,
 'valid_recall_at_0.9': 0.5965620066654973}

## Testing

In [15]:
trainer.test(ckpt_path='best', verbose=False)

[{'test_f1_at_0.7': 0.23192944000835028,
  'test_f1_at_0.8': 0.5442204574764135,
  'test_f1_at_0.9': 0.618759877289207,
  'test_pair_entity_ratio_at_0.7': 2.897340425531915,
  'test_pair_entity_ratio_at_0.8': 0.8061170212765958,
  'test_pair_entity_ratio_at_0.9': 0.4536347517730496,
  'test_precision_at_0.7': 0.13597699039226485,
  'test_precision_at_0.8': 0.4408885956230067,
  'test_precision_at_0.9': 0.6503810826656244,
  'test_recall_at_0.7': 0.7879432624113475,
  'test_recall_at_0.8': 0.7108156028368794,
  'test_recall_at_0.9': 0.5900709219858156}]